In [1]:
%load_ext cudf.pandas

<img src="https://docs.xarray.dev/en/stable/_static/dataset-diagram-logo.png" align="right" width="30%">

# Manipulating Dimensions (Data Resolution)

Sometimes we need to change the resolution of our data. We might need to look at inferred values between dimension (grid) spaces
or change the dimension spacing completely (for instance to add another variable). Learning goals:

- Interpolate data to new coordinates

In [2]:
%%cudf.pandas.profile

import numpy as np
import pandas as pd
import xarray as xr

np.random.seed(0)

                                                                                              
                                  Total time elapsed: 1.831 seconds                           
                                0 GPU function calls in 0.000 seconds                         
                                0 CPU function calls in 0.000 seconds                         
                                                                                              
                                                Stats                                         
                                                                                              
┏━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ Function ┃ GPU ncalls ┃ GPU cumtime ┃ GPU percall ┃ CPU ncalls ┃ CPU cumtime ┃ CPU percall ┃
┡━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
└──────────┴────────────┴─────────────┴─────────────┴────────────┴─────────────┴─────────────┘

In [3]:
%%cudf.pandas.profile

arr = xr.DataArray(
    np.random.randn(4, 6),
    dims=("x", "y"),
    coords={
        "x": [-3.2, 2.1, 5.3, 6.5],
        "y": pd.date_range("2009-01-05", periods=6, freq="M"),
    },
)
arr

<xarray.DataArray (x: 4, y: 6)>
array([[ 1.76405235,  0.40015721,  0.97873798,  2.2408932 ,  1.86755799,
        -0.97727788],
       [ 0.95008842, -0.15135721, -0.10321885,  0.4105985 ,  0.14404357,
         1.45427351],
       [ 0.76103773,  0.12167502,  0.44386323,  0.33367433,  1.49407907,
        -0.20515826],
       [ 0.3130677 , -0.85409574, -2.55298982,  0.6536186 ,  0.8644362 ,
        -0.74216502]])
Coordinates:
  * x        (x) float64 -3.2 2.1 5.3 6.5
  * y        (y) datetime64[ns] 2009-01-31 2009-02-28 ... 2009-05-31 2009-06-30

                                                                                                                
                                           Total time elapsed: 2.181 seconds                                    
                                        104 GPU function calls in 1.794 seconds                                 
                                        111 CPU function calls in 0.108 seconds                                 
                                                                                                                
                                                         Stats                                                  
                                                                                                                
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ Function                   ┃ GPU ncalls ┃ GPU cumtime ┃ GPU percall ┃ CPU ncalls ┃ CPU cumtime ┃ CPU percall ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ date_range                 │ 0          │ 0.000       │ 0.000       │ 1          │ 0.002       │ 0.002       │
│ Index                      │ 1          │ 1.745       │ 1.745       │ 0          │ 0.000       │ 0.000       │
│ Float64Index.__len__       │ 92         │ 0.034       │ 0.000       │ 0          │ 0.000       │ 0.000       │
│ DatetimeIndex.__len__      │ 0          │ 0.000       │ 0.000       │ 99         │ 0.092       │ 0.001       │
│ Float64Index.copy          │ 1          │ 0.000       │ 0.000       │ 0          │ 0.000       │ 0.000       │
│ Float64Index.__setattr__   │ 2          │ 0.001       │ 0.001       │ 0          │ 0.000       │ 0.000       │
│ DatetimeIndex.copy         │ 0          │ 0.000       │ 0.000       │ 1          │ 0.001       │ 0.001       │
│ DatetimeIndex.__setattr__  │ 0          │ 0.000       │ 0.000       │ 2          │ 0.002       │ 0.001       │
│ Float64Index.__deepcopy__  │ 1          │ 0.001       │ 0.001       │ 0          │ 0.000       │ 0.000       │
│ DatetimeIndex.__deepcopy__ │ 0          │ 0.000       │ 0.000       │ 1          │ 0.001       │ 0.001       │
│ Float64Index.__getitem__   │ 6          │ 0.006       │ 0.001       │ 0          │ 0.000       │ 0.000       │
│ DatetimeIndex.__getitem__  │ 0          │ 0.000       │ 0.000       │ 6          │ 0.007       │ 0.001       │
│ Float64Index.__repr__      │ 1          │ 0.007       │ 0.007       │ 0          │ 0.000       │ 0.000       │
│ DatetimeIndex.__repr__     │ 0          │ 0.000       │ 0.000       │ 1          │ 0.003       │ 0.003       │
└────────────────────────────┴────────────┴─────────────┴─────────────┴────────────┴─────────────┴─────────────┘

Not all pandas operations ran on the GPU. The following functions required CPU fallback:

- date_range
- DatetimeIndex.__len__
- DatetimeIndex.copy
- DatetimeIndex.__setattr__
- DatetimeIndex.__deepcopy__
- DatetimeIndex.__getitem__
- DatetimeIndex.__repr__

To request GPU support for any of these functions, please file a Github issue here: 
]8;id=970508;https://github.com/rapidsai/cudf/issues/new?assignees=&labels=%3F+-+Needs+Triage%2C+feature+request&projects=&template=pandas_function_request.md&title=%5BFEA%5D\https://github.com/rapidsai/cudf/issues/new/choose]8;;\.

## Interpolation

If we want to look at values between the current grid cells (interpolation), we
can do that with `interp` (requires `scipy`):


In [4]:
%%cudf.pandas.profile

arr.interp(
    x=np.linspace(2, 6, 10),
    y=pd.date_range("2009-04-01", "2009-04-30", freq="D"),
)

TypeError: <class 'pandas.core.indexes.datetimes.DatetimeIndex'>

                                                                                                           
                                         Total time elapsed: 0.427 seconds                                 
                                       3 GPU function calls in 0.003 seconds                               
                                       2 CPU function calls in 0.003 seconds                               
                                                                                                           
                                                       Stats                                               
                                                                                                           
┏━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ Function              ┃ GPU ncalls ┃ GPU cumtime ┃ GPU percall ┃ CPU ncalls ┃ CPU cumtime ┃ CPU percall ┃
┡━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ date_range            │ 0          │ 0.000       │ 0.000       │ 1          │ 0.002       │ 0.002       │
│ Float64Index.__len__  │ 2          │ 0.001       │ 0.000       │ 0          │ 0.000       │ 0.000       │
│ DatetimeIndex.__len__ │ 0          │ 0.000       │ 0.000       │ 1          │ 0.001       │ 0.001       │
│ Index                 │ 1          │ 0.003       │ 0.003       │ 0          │ 0.000       │ 0.000       │
└───────────────────────┴────────────┴─────────────┴─────────────┴────────────┴─────────────┴─────────────┘

Not all pandas operations ran on the GPU. The following functions required CPU fallback:

- date_range
- DatetimeIndex.__len__

To request GPU support for any of these functions, please file a Github issue here: 
]8;id=50353;https://github.com/rapidsai/cudf/issues/new?assignees=&labels=%3F+-+Needs+Triage%2C+feature+request&projects=&template=pandas_function_request.md&title=%5BFEA%5D\https://github.com/rapidsai/cudf/issues/new/choose]8;;\.

when trying to extrapolate, the resulting values will be `nan`.

If we already have a object with the desired coordinates, we can use
`interp_like`:


In [5]:
%%cudf.pandas.profile

other = xr.DataArray(
    dims=("x", "y"),
    coords={
        "x": np.linspace(2, 4, 10),
        "y": pd.date_range("2009-04-01", "2009-04-30", freq="D"),
    },
)
arr.interp_like(other)

<xarray.DataArray (x: 10, y: 30)>
array([[-6.52066740e-02, -4.76087761e-02, -3.00108783e-02,
        -1.24129804e-02,  5.18491747e-03,  2.27828153e-02,
         4.03807132e-02,  5.79786111e-02,  7.55765089e-02,
         9.31744068e-02,  1.10772305e-01,  1.28370203e-01,
         1.45968100e-01,  1.63565998e-01,  1.81163896e-01,
         1.98761794e-01,  2.16359692e-01,  2.33957590e-01,
         2.51555488e-01,  2.69153385e-01,  2.86751283e-01,
         3.04349181e-01,  3.21947079e-01,  3.39544977e-01,
         3.57142875e-01,  3.74740773e-01,  3.92338671e-01,
         4.09936568e-01,  4.27534466e-01,  4.45132364e-01],
       [-6.59905628e-02, -4.96577701e-02, -3.33249774e-02,
        -1.69921847e-02, -6.59391953e-04,  1.56734008e-02,
         3.20061935e-02,  4.83389862e-02,  6.46717789e-02,
         8.10045716e-02,  9.73373643e-02,  1.13670157e-01,
         1.30002950e-01,  1.46335742e-01,  1.62668535e-01,
         1.79001328e-01,  1.95334121e-01,  2.11666913e-01,
         2.27999706e-01,  2.44332499e-01,  2.60665291e-01,
         2.76998084e-01,  2.93330877e-01,  3.09663670e-01,
         3.25996462e-01,  3.42329255e-01,  3.58662048e-01,
         3.74994840e-01,  3.91327633e-01,  4.07660426e-01],
...
       [ 1.89840905e-01,  1.96062485e-01,  2.02284065e-01,
         2.08505645e-01,  2.14727225e-01,  2.20948806e-01,
         2.27170386e-01,  2.33391966e-01,  2.39613546e-01,
         2.45835126e-01,  2.52056706e-01,  2.58278287e-01,
         2.64499867e-01,  2.70721447e-01,  2.76943027e-01,
         2.83164607e-01,  2.89386188e-01,  2.95607768e-01,
         3.01829348e-01,  3.08050928e-01,  3.14272508e-01,
         3.20494088e-01,  3.26715669e-01,  3.32937249e-01,
         3.39158829e-01,  3.45380409e-01,  3.51601989e-01,
         3.57823569e-01,  3.64045150e-01,  3.70266730e-01],
       [ 2.26388257e-01,  2.31165378e-01,  2.35942500e-01,
         2.40719621e-01,  2.45496742e-01,  2.50273863e-01,
         2.55050985e-01,  2.59828106e-01,  2.64605227e-01,
         2.69382348e-01,  2.74159470e-01,  2.78936591e-01,
         2.83713712e-01,  2.88490833e-01,  2.93267955e-01,
         2.98045076e-01,  3.02822197e-01,  3.07599318e-01,
         3.12376440e-01,  3.17153561e-01,  3.21930682e-01,
         3.26707803e-01,  3.31484925e-01,  3.36262046e-01,
         3.41039167e-01,  3.45816288e-01,  3.50593410e-01,
         3.55370531e-01,  3.60147652e-01,  3.64924773e-01]])
Coordinates:
  * x        (x) float64 2.0 2.222 2.444 2.667 2.889 3.111 3.333 3.556 3.778 4.0
  * y        (y) datetime64[ns] 2009-04-01 2009-04-02 ... 2009-04-29 2009-04-30

                                                                                                                
                                           Total time elapsed: 0.974 seconds                                    
                                        219 GPU function calls in 0.099 seconds                                 
                                        176 CPU function calls in 0.175 seconds                                 
                                                                                                                
                                                         Stats                                                  
                                                                                                                
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ Function                   ┃ GPU ncalls ┃ GPU cumtime ┃ GPU percall ┃ CPU ncalls ┃ CPU cumtime ┃ CPU percall ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ date_range                 │ 0          │ 0.000       │ 0.000       │ 1          │ 0.001       │ 0.001       │
│ Index                      │ 4          │ 0.008       │ 0.002       │ 0          │ 0.000       │ 0.000       │
│ Float64Index.__len__       │ 160        │ 0.056       │ 0.000       │ 0          │ 0.000       │ 0.000       │
│ DatetimeIndex.__len__      │ 20         │ 0.006       │ 0.000       │ 150        │ 0.117       │ 0.001       │
│ Float64Index.copy          │ 8          │ 0.004       │ 0.001       │ 0          │ 0.000       │ 0.000       │
│ Float64Index.__setattr__   │ 2          │ 0.001       │ 0.000       │ 0          │ 0.000       │ 0.000       │
│ DatetimeIndex.copy         │ 2          │ 0.001       │ 0.000       │ 6          │ 0.005       │ 0.001       │
│ DatetimeIndex.__setattr__  │ 0          │ 0.000       │ 0.000       │ 2          │ 0.002       │ 0.001       │
│ Float64Index.__deepcopy__  │ 4          │ 0.003       │ 0.001       │ 0          │ 0.000       │ 0.000       │
│ DatetimeIndex.__deepcopy__ │ 0          │ 0.000       │ 0.000       │ 4          │ 0.004       │ 0.001       │
│ Float64Index.__getitem__   │ 9          │ 0.011       │ 0.001       │ 0          │ 0.000       │ 0.000       │
│ DatetimeIndex.__getitem__  │ 2          │ 0.002       │ 0.001       │ 7          │ 0.007       │ 0.001       │
│ Float64Index.set_names     │ 2          │ 0.001       │ 0.001       │ 0          │ 0.000       │ 0.000       │
│ Float64Index.get_indexer   │ 0          │ 0.000       │ 0.000       │ 2          │ 0.014       │ 0.007       │
│ ndarray.item               │ 4          │ 0.001       │ 0.000       │ 0          │ 0.000       │ 0.000       │
│ DatetimeIndex.set_names    │ 1          │ 0.001       │ 0.001       │ 1          │ 0.001       │ 0.001       │
│ DatetimeIndex.get_indexer  │ 0          │ 0.000       │ 0.000       │ 2          │ 0.020       │ 0.010       │
│ Float64Index.__repr__      │ 1          │ 0.004       │ 0.004       │ 0          │ 0.000       │ 0.000       │
│ DatetimeIndex.__repr__     │ 0          │ 0.000       │ 0.000       │ 1          │ 0.003       │ 0.003       │
└────────────────────────────┴────────────┴─────────────┴─────────────┴────────────┴─────────────┴─────────────┘

Not all pandas operations ran on the GPU. The following functions required CPU fallback:

- date_range
- DatetimeIndex.__len__
- DatetimeIndex.copy
- DatetimeIndex.__setattr__
- DatetimeIndex.__deepcopy__
- DatetimeIndex.__getitem__
- Float64Index.get_indexer
- DatetimeIndex.set_names
- DatetimeIndex.get_indexer
- DatetimeIndex.__repr__

To request GPU support for any of these functions, please file a Github issue here: 
]8;id=207518;https://github.com/rapidsai/cudf/issues/new?assignees=&labels=%3F+-+Needs+Triage%2C+feature+request&projects=&template=pandas_function_request.md&title=%5BFEA%5D\https://github.com/rapidsai/cudf/issues/new/choose]8;;\.